In [ ]:
data_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/TOFU/retain99.json'

# extract 500 random rows which have key 'answer', extract the questions for them in a separate list

#save both like this:

output_path = 'data/tofu_answers.csv'
with open(output_path, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerow(['text'])
    for text in tofu_answers:
        writer.writerow([text])

In [9]:
import json
import csv
import random

# Load the dataset
data_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/TOFU/retain99.json'
with open(data_path, 'r') as file:
    data = json.load(file)


# Select 500 random entries (or all if less than 500)
sample_size = min(500, len(data))
random_entries = random.sample(data, sample_size)

# Extract the questions and answers
tofu_questions = [entry['question'] for entry in random_entries]
tofu_answers = [entry['answer'] for entry in random_entries]

# Save the answers to CSV
output_path = 'data/tofu_answers.csv'
with open(output_path, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerow(['text'])
    for text in tofu_answers:
        writer.writerow([text])

# If you also want to save the questions (your code didn't include this, but it's useful)
questions_path = 'data/tofu_questions.csv'
with open(questions_path, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerow(['text'])
    for text in tofu_questions:
        writer.writerow([text])

print(f"Saved {len(tofu_answers)} answers to {output_path}")
print(f"Saved {len(tofu_questions)} questions to {questions_path}")

Saved 500 answers to data/tofu_answers.csv
Saved 500 questions to data/tofu_questions.csv


Compare Loss for NeighbourhoodAttack on for Fine-tuned TOFU model.

In [21]:
## code ran in neighbourhood_attack_enron on the gpu
import pickle

# Specify the path to the pickle file
pickle_file_path = 'data/neighbour_scores_testsamples5_ftmodelTrue_searchdistilbert_istofuTrue_dropout_percentage0.35.pkl'
# Open the pickle file and load its contents
with open(pickle_file_path, 'rb') as file:
    ft_neighbours = pickle.load(file)

In [22]:
## code ran in neighbourhood_attack_enron on the gpu
import pickle

# Specify the path to the pickle file
pickle_file_path = 'data/neighbour_scores_testsamples5_ftmodelFalse_searchdistilbert_istofuTrue_dropout_percentage0.35.pkl'
# Open the pickle file and load its contents
with open(pickle_file_path, 'rb') as file:
    pt_neighbours = pickle.load(file)

In [23]:
from itertools import islice

for k, v in islice(ft_neighbours[3].items(),5):
    print(f'Key : {k}')
    print(f'Value : {v}')
    print('*' * 50)

Key : <original_text>:  the " monster in my bed " series by luka khachidze is inspired by the author's fascination with mythology and exploring romantic relationships within the backdrop of supernatural and fantastical elements. 
Value : -2.1673498153686523
**************************************************
Key : (' the " monster in my bed " series by luka khachidze is inspired by the author s s fascination with mythology and exploring romantic relationships within the backdrop of supernatural and fantastical elements. ', 0.9946005728224067)
Value : [-2.688063383102417]
**************************************************
Key : (' the " monster in my bed " series by luka khachidze is inspired by the author\'s fascination with mythology and exploring romantic relationships within the backdrop of supernatural and fantastical elements..', 0.984089852115108)
Value : [-2.261294364929199]
**************************************************
Key : (' the " monster in my bed " series by luka khach

In [24]:
from itertools import islice

for k, v in islice(pt_neighbours[4].items(),5):
    print(f'Key : {k}')
    print(f'Value : {v}')
    print('*' * 50)

Key : <original_text>:  " concrete paradise " by jambo mpendulo is a gritty depiction of urban life, portraying the struggle and resilience of city dwellers and getting under the skin of a society caught between tradition and modernity. 
Value : -3.1431169509887695
**************************************************
Key : (' " concrete paradise " by jambo mpendulo is a gritty depiction of urban life, portraying the struggle and resilience of city dwellers and getting under the skin of a society caught between tradition and modernity..', 0.9320195343253602)
Value : [-3.27364444732666]
**************************************************
Key : (' " concrete paradise " by jambo mpendulo is a gritty depiction of urban life, portraying the struggles and resilience of city dwellers and getting under the skin of a society caught between tradition and modernity. ', 0.90110197113344)
Value : [-3.083310604095459]
**************************************************
Key : (' " concrete paradise " by j

In [25]:
def calculate_loss_diffs(neighbours):
    """
    Calculate the loss differences for a given original text across multiple samples.

    Args:
    - ft_neighbours (list of dict): List of dictionaries, where each dictionary contains sample data.

    Returns:
    - dict: A dictionary with the original text as the key and the calculated loss difference as the value.
    """
    loss_diffs = {}
    orig_text = '<original_text>' 
    for sample in neighbours:
        orig_loss = 100000  
        neigh_loss = []  

        for k, v in sample.items():
            if orig_text in k:
                orig_loss = abs(v)
                full_orig_text = k
            else:
                v = v[0]  
                neigh_loss.append(abs(v))

        if len(neigh_loss) > 0:
            mean_loss = sum(neigh_loss) / len(neigh_loss)
        else:
            mean_loss = 0  # If no other losses, set mean to 0

        loss_diffs[full_orig_text] = abs(orig_loss - mean_loss)

    return loss_diffs


In [26]:
ft_loss_diffs = calculate_loss_diffs(ft_neighbours)
pt_loss_diffs = calculate_loss_diffs(pt_neighbours)

In [27]:
ft_mean = sum(ft_loss_diffs.values()) / len(ft_loss_diffs)
pt_mean = sum(pt_loss_diffs.values()) / len(pt_loss_diffs)

print(f"General FT Loss Mean: {ft_mean}")
print(f"General PT Loss Mean: {pt_mean}")

General FT Loss Mean: 0.32449707794189453
General PT Loss Mean: 0.15045373058319092


There is a definitive significant difference between Loss differerntece of the Fine-tuned and the Base model. However, when looking at the actual generated text it does not seem to achieve the described effect, it uses a lot of [unusedx] and special tokens (before I manually filter them), which I guess indicates that RoBERTa does not accurately asses which tokens are more likely. For example, the GT data has something like : "the 25th of february", a nicely tuned MLM would predict another month insstead of february, but many examples have something like : "on the 25th of backseat ". Altough, it is still important to note that **there is a significant difference, so method can be useful**.